In [1]:
import csv
import PyPDF2
import pandas as pd
import textract
import sys, re, os
import random

In [2]:
def napraviCSV(path, pisac, tip_fajla):
    #ucitavamo pdf fajl i citamo tekst tog pdf fajla
    text = textract.process(path, method='pdfminer', encoding = 'raw_unicode_escape')

    text = text.decode('raw_unicode_escape')
    
    #brisemo entere     
    ri = re.compile(r"\n")
    newText = re.sub(ri, r' ', text)
    
    ri = re.compile(r"\t")
    newText = re.sub(ri, r' ', newText.strip('\t'))
    
    ri = re.compile(r"\ñ")
    newText = re.sub(ri, r'dj', newText)

    #brisemo znakove intepunkcije
    ri = re.compile(r"[!.\?<=>:;,_-]")
    newText = re.sub(ri, r'', newText)
    
    ri = re.compile(r"[\—\»\«\(\)]")
    newText = re.sub(ri, r'', newText)
    
    #ubacujemo slovo ž
    ri = re.compile(r"(\\u017)|[žŽ]")
    newText = re.sub(ri, r'z', newText)

    #ubacujemo slovo ć
    ri = re.compile(r"(\\u0107)|[ćĆ]")
    newText = re.sub(ri, r'c', newText)

    #ubacujemo slovo č
    ri = re.compile(r"(\\u010d)|[čČ]")
    newText = re.sub(ri, r'c', newText)

    #ubacujemo slovo đ
    ri = re.compile(r"(\\u0161)|[đĐ]")
    newText = re.sub(ri, r'dj', newText)
    
    ri = re.compile(r" +")
    newText = re.sub(ri, r' ', newText)

    #ubacujemo slovo š
    ri = re.compile(r"(\/xfl)|[šŠ]")
    krajnji = re.sub(ri, r's', newText)
    
    #ubacujemo newText u listu
    for f in re.findall("([A-Z]+)", krajnji):
        #prebacujemo sva velika slova u mala
        krajnji = krajnji.replace(f, f.lower())
    #string krajnji razdvajamo u listu, separator je razmak
    lista = krajnji.split(' ')

    print(len(lista))
    #4897874
    tmp = ""
    #u ovu promenjivu cemo da stavimo sve redove koje ubacujemo u CSV
    listaSemplova = []

    pocetak = len(lista) % 1500
        
    for i in range(pocetak, len(lista)):
        tmp += lista[i] + " "
        if(i % 500 == 0 and i > 0):
            listaSemplova.append([tmp, pisac])
            tmp = ""
    
    if(tip_fajla == "test" and len(listaSemplova) > 100):
        listaSemplova = random.sample(listaSemplova, 100)
        
    my_df = pd.DataFrame(listaSemplova)
    #my_df.to_csv(pisac+'.csv', index=False, header=False)
    if(tip_fajla == "trening"):
        print(my_df.head())
        #proveravamo da li postoji CSV fajl i ako postoji appendujemo, ako ne pravimo ga.
        if not os.path.isfile('trening.csv'):
            my_df.to_csv('trening.csv', header='column_names', index=False)
        else: # else it exists so append without writing the header
            my_df.to_csv('trening.csv', mode='a', header=False, index=False)
    else:
        print("USAO OVDE ZA KNJIGU ")
        print(path)
        print(len(my_df))
        #proveravamo da li postoji CSV fajl i ako postoji appendujemo, ako ne pravimo ga.
        if not os.path.isfile('test.csv'):
            my_df.to_csv('test.csv', header='column_names', index=False)
        else: # else it exists so append without writing the header
            my_df.to_csv('test.csv', mode='a', header=False, index=False)

In [3]:
path = 'knjige/Ivo Andrić - Travnička hronika.pdf'
napraviCSV(path, "andric", "trening")

147578
                                                   0       1
0  recnik turskih reci i provincijalizama 1 2 za...  andric
1  mu duga stanka vojske su ovdje padale pa se ni...  andric
2  dok sja nigde ne sja tako kao nad njihovim gra...  andric
3  stranac ugledan putnik mozda i to se zna sta j...  andric
4  u srbiji kao vidan znak novih vremena i novih ...  andric


In [4]:
path = 'knjige/Ivo-Andric-Na-Drini-cuprija.pdf'
napraviCSV(path, "andric", "trening")

114877
                                                   0       1
0  zivot coveka iz kasabe u svima pricanjima o li...  andric
1  znali su sve majstorski izrezane obline i udub...  andric
2  ga jos nije videlo jer deca ne umiru ali ga je...  andric
3  zovu bunarima i u njima drze i jedni i drugi b...  andric
4  dobacivanja i sasaptavanja tu su i prvi poslov...  andric


In [5]:
path = 'knjige/Meša Selimović - Derviš i smrt.pdf'
napraviCSV(path, "selimovic", "trening")

119180
                                                   0          1
0  to samo rijec ili je misao ako je misao onda j...  selimovic
1  da ima mnogo slicnosti izmedju tog stjesnjavan...  selimovic
2  sve ima vremena dao sam ga sam sebi a sudjenje...  selimovic
3  vidim ono sto me se ne tice tako sam blize seb...  selimovic
4  on tobom zadovolјan udji medju robove moje udj...  selimovic


In [6]:
path = 'knjige/mesa-selimovic-tvrdjava.pdf'
napraviCSV(path, "selimovic", "trening")

108628
                                                   0          1
0  pratilackim pojavama uskomesanosti bunta i ner...  selimovic
1  preplivao sam od kisa nabujali dnjestar i tako...  selimovic
2  vojnicima u svakome od nas za drugoga je bilo ...  selimovic
3  osvajalo je nesto sto im nije bilo potrebno i ...  selimovic
4  o kravi sama muz joj je sigurno s onu stranu m...  selimovic


In [7]:
path = 'testKnjige/Ivo Andric Gospodjica.pdf'
napraviCSV(path, "andric", "test")

69123
USAO OVDE ZA KNJIGU 
testKnjige/Ivo Andric Gospodjica.pdf
100


In [8]:
path = 'testKnjige/Meša Selimović - Tišine.pdf'
napraviCSV(path, "selimovic", "test")

50672
USAO OVDE ZA KNJIGU 
testKnjige/Meša Selimović - Tišine.pdf
99


In [9]:
trening = pd.read_csv('trening.csv')
print(len(trening))
test = pd.read_csv('test.csv')
print(len(test))

1950
199
